Copyright (c) 2023 Graphcore Ltd. All rights reserved.

# DFT dataset generation using PySCF IPU

This notebook shows how to generate DFT datasets on the Graphcore IPU based on
"PySCFIPU: Repurposing Density Functional Theory to Suit Deep Learning", Mathiasen et al, (SynS & ML) Workshop, ICML 2023

https://icml.cc/virtual/2023/28485

Density Functional Theory (DFT) accurately predicts the properties of molecules given their atom types and positions,
and often serves as ground truth for molecular property prediction tasks.
Research in other areas of machine learning has shown that generalisation performance 
of Neural Networks tends to improve with increased dataset size, however, 
the computational cost of DFT has limited the size of DFT datasets.

PySCF_IPU allowed us to create QM10X, a dataset with 100 million conformers, in 3000 IPU-hours.

This notebook, running on 4 IPUs on Paperspace, creates 100K conformers in about 1 hour;
it can be run on multiple 16-IPU systems (paid instances) to generate the full dataset.

Install `pyscf-ipu`:

In [2]:
# PySCF IPU dependencies 
%pip install -e "..[ipu]"
print('install done')

Obtaining file:///notebooks
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 34.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.3/111.3 MB 51.2 MB/s eta 0:00:0000:0100:01
  Cloning https://github.com/graphcore-research/tessellate-ipu.git (to revision main) to /tmp/pip-install-v1ttl504/tessellate-ipu_324d33ed0e4d4c839458b5dee8c8ace7
  Running command git clone --filter=blob:none --quiet https://github.com/graphcore-research/tessellate-ipu.git /tmp/pip-install-v1ttl504/tessellate-ipu_324d33ed0e4d4c839458b5dee8c8ace7
  Resolved https://github.com/graphcore-research/tessellate-ipu.git to commit e254e597bc9b2ffdfa6e3f6d814bebde9bd2f716
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Attempting uninstall: pyscf-ipu
    Found existing installation: pyscf-ipu 0.0.1
    Can't uninstall 'pyscf-

# Download and preprocess GDB 11 dataset

In [2]:
import os

gdb_filename = "./data/gdb11_size09.smi"
out_filename = gdb_filename.replace(".smi", "_sorted.csv")

loaded = os.path.exists(out_filename) and os.path.getsize(out_filename) == 6985727

if loaded:
  print(f'Found {out_filename}')
else:
  print(f'Did not find {out_filename}, or size was wrong')

  # Download and extract GDB11 dataset.
  !mkdir -p ./data
  !wget -p -O ./data/gdb11.tgz https://zenodo.org/record/5172018/files/gdb11.tgz\?download\=1
  !tar -xvf ./data/gdb11.tgz --directory ./gdb/

  from  gdb import sortgdb

  # Filter & sort GDB11 dataset (size 9).
  gdb_sorted = sortgdb.sort_gdb(gdb_filename, keep_only_atoms_count=9)
  # Save output as csv.
  gdb_sorted.to_csv(out_filename, index=False, header=False)



Found ./data/gdb11_size09_sorted.csv


In [3]:
import os

# PySCF IPU setup: use a single device per process.
os.environ["JAX_IPU_DEVICE_COUNT"] = "1"
# JAX/XLA IPU compilation cache.
os.environ['TF_POPLAR_FLAGS'] = """
  --executable_cache_path=/tmp/ipu-ef-cache
"""

# First import of JAX and TessellateIPU may take a few minutes...
import jax
import tessellate_ipu
print('import done')

import done


# Create a DFT dataset using PySCF IPU

In the following example, we use only a single IPU. Multiple IPUs can be used by simply launching a collection of PySCF IPU processes instead of a single one.

In [4]:
# Equivalent to command line:
# python pyscf_ipu/dft.py  -generate  -save  -fname "notebook_dataset"
#        -level 0  -plevel 0  -num_conformers 1000
#        -gdb 9  -float32

import time
from pyscf_ipu.dft import get_args, process_args

args = get_args([])
args.backend = 'cpu'
args.generate = True
args.save = True
args.fname = "notebook_dataset"

args.level = 0
args.plevel = 0

args.float32 = True

quick = True # Set to False to generate full dataset (takes some time)

if quick:
  args.id = 1
  args.num_conformers = 32 # Set to 1000 for full dataset
  args.limit = 33 # Comment out for full dataset
else:
  args.num_conformers = 1000

process_args(args)

# Load GDB09 data
gdb = 'gdb11_size09'
args.smiles = open(f"../gdb/{gdb}_sorted.csv", "r").read().split("\n")
print(f'Loaded {len(args.smiles)} molecules from {gdb}')

[BASIS] STO-3G
Loaded 444285 molecules from gdb11_size09


In [ ]:
from rdkit import Chem

from pyscf_ipu.dft import angstrom_to_bohr, get_atom_string, jax_dft, recompute

print("Length GDB: ", len(args.smiles))

if args.limit != -1:
    args.smiles = args.smiles[:args.limit]

for i in range(int(args.id), min(int(args.id)+1000, len(args.smiles))):
    smile = args.smiles[i]

    print('Trying', smile)

    b = Chem.MolFromSmiles(smile)
    b = Chem.AddHs(b, explicitOnly=False)

    e = Chem.AllChem.EmbedMolecule(b)
    if e == -1:
       print('Did not embed', b) 
       continue

    locs = b.GetConformer().GetPositions() * angstrom_to_bohr
    atoms = [atom.GetSymbol() for atom in b.GetAtoms()]
    atom_string, string = get_atom_string(" ".join(atoms), locs)

    print('Conformer: ', string)
    break

recompute(args, None, 0, 0, our_fun=jax_dft, str=string)

Length GDB:  444285
Trying FC(F)=C(F)C#CC#N
Conformer:  F  -3.303279 -2.551231 -0.665592; C  -3.551114 -0.180504 -1.594233; F  -5.642254 0.510763 -2.775206; C  -1.480470 1.359424 -1.330889; F  -1.829597 3.736696 -2.292413; C  0.721395 0.525248 -0.394711; C  2.921869 -0.329714 0.586346; C  5.124244 -1.156335 1.526403; N 7.039206 -1.914347 2.335843; 
recompute: _0_32
HIT
0_GDB-1_f32True_grid0_backendcpu_0_32
(146, 16)
1_GDB-1_f32True_grid0_backendcpu_0_32


  0%|          | 0/33 [00:00<?, ?it/s]

[PAD] Last molecule had grisize=9816 we're using 10797. 
[FC(F)(C#N)C(=O)C#N]
[conformers] 32
16.4025 (45, 45, 45, 45) 15
7.77384 (4, 10797, 45) 20


[1 / 32] Hs=    0 -529.163500 4513.4 0.0 0.0 0.1 3.0 0.2 298.2 0.1 28.0 4.4 0.3 0.3 0.1 0.0 0.0 3.3 0.2 0.0 0.0 0.2 0.3 1.1 0.2 4.5 4857.9 [1 ; 0]:   0%|          | 0/33 [00:05<?, ?it/s]

16.4025 (45, 45, 45, 45) 15
7.77384 (4, 10797, 45) 20


...

16.4025 (45, 45, 45, 45) 15
7.77384 (4, 10797, 45) 20


[16 / 32] Hs=    0 -606.347493 4477.4 0.1 0.0 0.1 3.0 0.2 297.0 0.0 26.5 3.6 0.2 0.3 0.1 0.0 0.0 3.1 0.2 0.0 0.0 0.2 0.3 0.6 0.1 1.9 4814.9 [21 ; 0]:  61%|██████    | 20/33 [53:08<34:02, 157.09s/it]

16.4025 (45, 45, 45, 45) 15
7.77384 (4, 10797, 45) 20


[17 / 32] Hs=    0 -606.333237 4722.6 0.5 0.0 0.1 3.5 0.3 288.5 0.1 27.4 2.1 0.3 0.3 0.2 0.0 0.0 3.3 0.2 0.0 0.0 0.2 0.4 2.0 0.3 4.9 5057.2 [21 ; 0]:  61%|██████    | 20/33 [53:13<34:02, 157.09s/it]

16.4025 (45, 45, 45, 45) 15
7.77384 (4, 10797, 45) 20


# Loading & visualizing generated data

After the dataset has been created, we can load the data.
(You may wish to spin up a new notebook, and view the data as 
it's being generated in this one).

In [ ]:
import pandas as pd

In [13]:
# Output DFT dataset is a compressed CSV file.
# NOTE: it may take a couple of minutes before the file is generated.
rootpath = f'./data/generated/{args.fname}/'
paths = sorted(os.listdir(rootpath), key=lambda x: os.path.getmtime(rootpath + x))
filename = os.path.join(rootpath, paths[-1], "data.csv")

df = pd.read_csv(filename, compression="gzip")

In [14]:
df

,Unnamed: 0,smile,atoms,atom_positions,energies,std,pyscf_energies,pyscf_hlgap,pyscf_homo,pyscf_lumo,times,homo,lumo,hlgap,N,basis
0,0,FC(F)(C#N)C(=O)C#N,FCFCNCOCN,"[0.15332033500393688, -2.3606980417798886, -2....","[-14387.150902075824, -14321.477650378294, -14...",0.000685,[0.0],0,0,0,[4.5198e+03 0.0000e+00 0.0000e+00 1.0000e-01 3...,-5.872426,-0.872470,4.999956,45,STO-3G
1,0,FC(F)(C#N)C(=O)C#N,FCFCNCOCN,"[-2.3403320213763945, -1.054070549382328, 2.95...","[-14386.940987492488, -14327.730510746002, -14...",0.001410,[0.0],0,0,0,[4.3918e+03 0.0000e+00 1.0000e-01 1.0000e-01 3...,-5.924247,-1.223614,4.700633,45,STO-3G
2,0,FC(F)(C#N)C(=O)C#N,FCFCNCOCN,"[-2.508312250101616, -2.9848902777921023, 0.05...","[-14386.867679381283, -14319.960709000135, -14...",0.001749,[0.0],0,0,0,[4.4863e+03 0.0000e+00 1.0000e-01 1.0000e-01 3...,-5.924836,-1.235037,4.689799,45,STO-3G
3,0,FC(F)(C#N)C(=O)C#N,FCFCNCOCN,"[-3.2551803300125353, -2.1329334256170753, 0.8...","[-14387.076018358, -14329.75193126052, -14398....",0.001727,[0.0],0,0,0,[4.4778e+03 1.0000e-01 0.0000e+00 1.0000e-01 3...,-5.899619,-1.169639,4.729980,45,STO-3G
4,0,FC(F)(C#N)C(=O)C#N,FCFCNCOCN,"[-2.84609364909322, -2.7599543764642225, -0.10...","[-14386.926915515363, -14322.835535730032, -14...",0.001680,[0.0],0,0,0,[4.3595e+03 0.0000e+00 0.0000e+00 1.0000e-01 3...,-5.838394,-1.148967,4.689428,45,STO-3G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,0,O=c1nc(nno1)C#N,OCNCNNOCN,"[-4.975797037092851, -0.10223483890282797, 1.6...","[-12967.214870320147, -12844.203558009567, -12...",0.000489,[0.0],0,0,0,[4.4596e+03 1.0000e-01 0.0000e+00 1.0000e-01 2...,-5.406635,-2.607664,2.798971,45,STO-3G
142,0,O=c1nc(nno1)C#N,OCNCNNOCN,"[-1.6667435103637371, 4.803784367677628, -1.16...","[-12967.173270446034, -12841.313417319576, -12...",0.001113,[0.0],0,0,0,[4.4013e+03 0.0000e+00 0.0000e+00 1.0000e-01 2...,-5.428957,-2.577853,2.851104,45,STO-3G
143,0,O=c1nc(nno1)C#N,OCNCNNOCN,"[-3.565835428561025, 1.1894789789949995, -3.62...","[-12967.267687429836, -12840.808835446367, -12...",0.001271,[0.0],0,0,0,[4.5021e+03 1.0000e-01 0.0000e+00 1.0000e-01 2...,-5.434490,-2.590510,2.843980,45,STO-3G
144,0,O=c1nc(nno1)C#N,OCNCNNOCN,"[-2.3093834500910693, -4.5080600038703045, -1....","[-12967.429002059404, -12842.168408659896, -12...",0.000779,[0.0],0,0,0,[4.5073e+03 0.0000e+00 0.0000e+00 1.0000e-01 2...,-5.460477,-2.586699,2.873778,45,STO-3G


In [15]:
# HLgap data.
df["hlgap"]

0      4.999956
1      4.700633
2      4.689799
3      4.729980
4      4.689428
         ...   
141    2.798971
142    2.851104
143    2.843980
144    2.873778
145    2.948429
Name: hlgap, Length: 146, dtype: float64